In [1]:
import getpass
import textwrap
import torch

from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer("intfloat/multilingual-e5-base", device=device)

In [2]:
print(device)

cpu


In [4]:
ELASTCSEARCH_CERT_PATH = "/Users/liuxg/elastic/elasticsearch-8.10.0/config/certs/http_ca.crt"
 
client = Elasticsearch(  ['https://localhost:9200'],
    basic_auth = ('elastic', 'vXDWYtL*my3vnKY9zCfL'),
    ca_certs = ELASTCSEARCH_CERT_PATH,
    verify_certs = True)
print(client.info())

{'name': 'liuxgm.local', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'lM5ZgEQNTkO8phtT5qZAHQ', 'version': {'number': '8.10.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'e338da74c79465dfdc204971e600342b0aa87b6b', 'build_date': '2023-09-07T08:16:21.960703010Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [5]:
# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "language": {"type": "keyword"},
            "id": {"type": "keyword"},
            "title": {"type": "text"},
            "passage": {"type": "text"},
            "passage_embedding": {
                "type": "dense_vector",
                "dims": 768,
                "index": "true",
                "similarity": "cosine"
            }
        }
    }
}

# Create the index (deleting any existing index)
client.indices.delete(index="articles", ignore_unavailable=True)
client.indices.create(index="articles", body=mapping)

/var/folders/kw/pvmj1zgs44l8t32cs8blf6sc0000gn/T/ipykernel_36180/3293026579.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(index="articles", body=mapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'articles'})

In [6]:
articles = [
    {
        "language": "en",
        "id": "1643584#0",
        "title": "Bloor Street",
        "passage": """Bloor Street is a major east–west residential and commercial thoroughfare in Toronto, Ontario, Canada. Bloor Street runs from the Prince Edward Viaduct, which spans the Don River Valley, westward into Mississauga where it ends at Central Parkway. East of the viaduct, Danforth Avenue continues along the same right-of-way. The street, approximately long, contains a significant cross-sample of Toronto's ethnic communities. It is also home to Toronto's famous shopping street, the Mink Mile.""",
    },
    {
        "language": "en",
        "id": "2190499#0",
        "title": "Elphinstone College",
        "passage": """Elphinstone College is an institution of higher education affiliated to the University of Mumbai. Established in 1856, it is one of the oldest colleges of the University of Mumbai. It is reputed for producing luminaries like Bal Gangadhar Tilak, Bhim Rao Ambedkar, Virchand Gandhi, Badruddin Tyabji, Pherozshah Mehta, Kashinath Trimbak Telang, Jamsetji Tata and for illustrious professors that includes Dadabhai Naoroji. It is further observed for having played a key role in spread of Western education in the Bombay Presidency.""",
    },
    {
        "language": "en",
        "id": "8881#0",
        "title": "Doctor (title)",
        "passage": """Doctor is an academic title that originates from the Latin word of the same spelling and meaning. The word is originally an agentive noun of the Latin verb "" 'to teach'. It has been used as an academic title in Europe since the 13th century, when the first Doctorates were awarded at the University of Bologna and the University of Paris. Having become established in European universities, this usage spread around the world. Contracted "Dr" or "Dr.", it is used as a designation for a person who has obtained a Doctorate (e.g. PhD). In many parts of the world it is also used by medical practitioners, regardless of whether or not they hold a doctoral-level degree.""",
    },
    {
        "language": "de",
        "id": "9002#0",
        "title": "Gesundheits- und Krankenpflege",
        "passage": """Die Gesundheits- und Krankenpflege als Berufsfeld umfasst die Versorgung und Betreuung von Menschen aller Altersgruppen, insbesondere kranke, behinderte und sterbende Erwachsene. Die Gesundheits- und Kinderkrankenpflege hat ihren Schwerpunkt in der Versorgung von Kindern und Jugendlichen. In beiden Fachrichtungen gehört die Verhütung von Krankheiten und Gesunderhaltung zum Aufgabengebiet der professionellen Pflege.""",
    },
    {
        "language": "de",
        "id": "7769762#0",
        "title": "Tourismusregion (Österreich)",
        "passage": """Unter Tourismusregion versteht man in Österreich die in den Landestourismusgesetzen verankerten Tourismusverbände mehrerer Gemeinden, im weiteren Sinne aller Gebietskörperschaften.""",
    },
    {
        "language": "de",
        "id": "2270104#0",
        "title": "London Wall",
        "passage": """London Wall ist die strategische Stadtmauer, die die Römer um Londinium gebaut haben, um die Stadt zu schützen, die über den wichtigen Hafen an der Themse verfügte. Bis ins späte Mittelalter hinein bildete diese Stadtmauer die Grenzen von London. Heute ist "London Wall" auch der Name einer Straße, die an einem noch bestehenden Abschnitt der Stadtmauer verläuft.""",
    },
    {
        "language": "de",
        "id": "2270104#1",
        "title": "London Wall",
        "passage": """Die Mauer wurde Ende des zweiten oder Anfang des dritten Jahrhunderts erbaut, wahrscheinlich zwischen 190 und 225, vermutlich zwischen 200 und 220. Sie entstand somit etwa achtzig Jahre nach dem im Jahr 120 erfolgten Bau der Festung, deren nördliche und westliche Mauern verstärkt und in der Höhe verdoppelt wurden, um einen Teil der neuen Stadtmauer zu bilden. Die Anlage wurde zumindest bis zum Ende des vierten Jahrhunderts weiter ausgebaut. Sie zählt zu den letzten großen Bauprojekten der Römer vor deren Rückzug aus Britannien im Jahr 410.""",
    },
]

In [7]:
actions = []
for article in articles:
    actions.append({"index": {"_index": "articles"}})
    passage = article["passage"]
    passageEmbedding = model.encode(f"passage: {passage}").tolist()
    article["passage_embedding"] = passageEmbedding
    actions.append(article)

client.bulk(index="articles", operations=actions)

ObjectApiResponse({'errors': False, 'took': 29, 'items': [{'index': {'_index': 'articles', '_id': 'Ub9jCIsByaLf0EuTgc0y', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'articles', '_id': 'Ur9jCIsByaLf0EuTgc0y', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'articles', '_id': 'U79jCIsByaLf0EuTgc0y', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'articles', '_id': 'VL9jCIsByaLf0EuTgc0y', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'articles', '_id': 'Vb9jCIsByaLf0EuTgc0y', '_version': 1, 'result': 'created', '_shards': {'total

In [8]:
def pretty_response(response):
    for hit in response["hits"]["hits"]:
        score = hit["_score"]
        language = hit["_source"]["language"]
        id = hit["_source"]["id"]
        title = hit["_source"]["title"]
        passage = hit["_source"]["passage"]
        print()
        print(f"ID: {id}")
        print(f"Language: {language}")
        print(f"Title: {title}")
        print(f"Passage: {textwrap.fill(passage, 120)}")
        print(f"Score: {score}")

In [9]:
def query(q, language=None):
    knn = {
        "field": "passage_embedding",
        "query_vector" : model.encode(f"query: {q}").tolist(),
        "k": 2,
        "num_candidates": 5
    }

    if language:
        knn["filter"] = {
            "term": {
                "language": language,
            }
        }

    return client.search(index="articles", knn=knn)

In [10]:
pretty_response(query("health"))


ID: 9002#0
Language: de
Title: Gesundheits- und Krankenpflege
Passage: Die Gesundheits- und Krankenpflege als Berufsfeld umfasst die Versorgung und Betreuung von Menschen aller Altersgruppen,
insbesondere kranke, behinderte und sterbende Erwachsene. Die Gesundheits- und Kinderkrankenpflege hat ihren Schwerpunkt
in der Versorgung von Kindern und Jugendlichen. In beiden Fachrichtungen gehört die Verhütung von Krankheiten und
Gesunderhaltung zum Aufgabengebiet der professionellen Pflege.
Score: 0.8986235

ID: 8881#0
Language: en
Title: Doctor (title)
Passage: Doctor is an academic title that originates from the Latin word of the same spelling and meaning. The word is originally
an agentive noun of the Latin verb "" 'to teach'. It has been used as an academic title in Europe since the 13th
century, when the first Doctorates were awarded at the University of Bologna and the University of Paris. Having become
established in European universities, this usage spread around the world. Contract

In [11]:
pretty_response(query("wall", language="de"))


ID: 2270104#0
Language: de
Title: London Wall
Passage: London Wall ist die strategische Stadtmauer, die die Römer um Londinium gebaut haben, um die Stadt zu schützen, die über
den wichtigen Hafen an der Themse verfügte. Bis ins späte Mittelalter hinein bildete diese Stadtmauer die Grenzen von
London. Heute ist "London Wall" auch der Name einer Straße, die an einem noch bestehenden Abschnitt der Stadtmauer
verläuft.
Score: 0.8941857

ID: 2270104#1
Language: de
Title: London Wall
Passage: Die Mauer wurde Ende des zweiten oder Anfang des dritten Jahrhunderts erbaut, wahrscheinlich zwischen 190 und 225,
vermutlich zwischen 200 und 220. Sie entstand somit etwa achtzig Jahre nach dem im Jahr 120 erfolgten Bau der Festung,
deren nördliche und westliche Mauern verstärkt und in der Höhe verdoppelt wurden, um einen Teil der neuen Stadtmauer zu
bilden. Die Anlage wurde zumindest bis zum Ende des vierten Jahrhunderts weiter ausgebaut. Sie zählt zu den letzten
großen Bauprojekten der Römer vor der

In [12]:
pretty_response(query("Mauer", language="de"))


ID: 2270104#1
Language: de
Title: London Wall
Passage: Die Mauer wurde Ende des zweiten oder Anfang des dritten Jahrhunderts erbaut, wahrscheinlich zwischen 190 und 225,
vermutlich zwischen 200 und 220. Sie entstand somit etwa achtzig Jahre nach dem im Jahr 120 erfolgten Bau der Festung,
deren nördliche und westliche Mauern verstärkt und in der Höhe verdoppelt wurden, um einen Teil der neuen Stadtmauer zu
bilden. Die Anlage wurde zumindest bis zum Ende des vierten Jahrhunderts weiter ausgebaut. Sie zählt zu den letzten
großen Bauprojekten der Römer vor deren Rückzug aus Britannien im Jahr 410.
Score: 0.88160384

ID: 2270104#0
Language: de
Title: London Wall
Passage: London Wall ist die strategische Stadtmauer, die die Römer um Londinium gebaut haben, um die Stadt zu schützen, die über
den wichtigen Hafen an der Themse verfügte. Bis ins späte Mittelalter hinein bildete diese Stadtmauer die Grenzen von
London. Heute ist "London Wall" auch der Name einer Straße, die an einem noch bestehe